In [1]:
from HLAGuessr.processing import Processing
from HLAGuessr.test_model import CrossValidation
import os

In [2]:
chain = ['alpha','beta']
delimiter = '\t'
hla_threshold=1
main_path = os.path.join(os.getcwd(),'HLAGuessr/')
grouping = True
untyped = False

#### Cross-validating classifier with typed dataset

In [3]:
alpha_input = main_path + 'Example_validation_data/alpha_all_files.tsv.gz'
beta_input = main_path + 'Example_validation_data/beta_all_files.tsv.gz'
hla_input = main_path + 'Example_validation_data/HLA_grouped_patients.tsv'
hla_target = 'A*02:01'
solver = 'liblinear'
penalty = 'l1'

In [4]:
dm = Processing(chain,alpha_input,beta_input,delimiter,grouping)

In [6]:
cv = CrossValidation(hla_target,hla_input,chain,dm.ps,dm.data_test,grouping,hla_threshold,solver,penalty,untyped)

100%|█████████████████████████████████████████| 21/21 [00:00<00:00, 3303.34it/s]
8it [00:00, 101.72it/s]
100%|█████████████████████████████████████████| 21/21 [00:00<00:00, 2626.21it/s]
8it [00:00, 161.78it/s]


Fitting 5 folds for each of 53 candidates, totalling 265 fits


In [6]:
cv.df_summary

,HLA,AUC,accuracy,precision,sensitivity,specificity,chain
0,A*02:01,0.982143,0.869565,0.772727,0.944444,0.821429,alpha+beta


#### Type individual from input repertoire

In [3]:
from HLAGuessr.load_model import PreprocessedModel
from HLAGuessr.evaluate_data import GetProbabilities
import HLAGuessr.model_tools as mt
import pandas as pd, numpy as np

In [4]:
alpha_input = main_path + 'Example_validation_data/alpha_example.tsv'
alpha_input = None
beta_input = main_path + 'Example_validation_data/beta_example.tsv'
grouping = False
untyped = True

In [5]:
dm = Processing(chain,alpha_input,beta_input,delimiter,grouping)

In [6]:
f_params = main_path + 'Training_data/classifier_params_alpha+beta.tsv'
df_param = pd.read_csv(f_params,delimiter='\t')
hla_set = list(df_param['HLA'])

In [7]:
hla_target = 'A*02:01'
df1 = df_param.loc[df_param['HLA']==hla_target,:]
auc = float(df1['AUC'].iloc[0])    

In [8]:
auc

0.9782608695652176

In [13]:
df_final_params = pd.DataFrame()
    
non_empty = [p for p in dm.ps if p]
unique_ps = list({tuple(p) for p in non_empty})
unique_ps = [list(p) for p in unique_ps]

for p in unique_ps:
    print(p)
            
    print('Individual ' +p[0]+ ':')

    for hla_target in hla_set:

        ev = GetProbabilities(hla_target,chain,hla_threshold,dm.ps,dm.data_test,grouping,untyped)
        params = ev.classifier_params(df_param,hla_target)
        df = mt.print_params(hla_target,ev.hla_prob,params,p[0],ev.hla_output)
        df_final_params = pd.concat([df,df_final_params],ignore_index=True)

['17_B']
Individual 17_B:


100%|█████████████████████████████████████████| 14/14 [00:00<00:00, 2298.88it/s]

100%|█████████████████████████████████████████| 13/13 [00:00<00:00, 2318.18it/s]

100%|█████████████████████████████████████████| 14/14 [00:00<00:00, 2664.98it/s]

100%|█████████████████████████████████████████| 14/14 [00:00<00:00, 2546.85it/s]
8it [00:00, 189.21it/s]
100%|█████████████████████████████████████████| 14/14 [00:00<00:00, 1000.99it/s]
8it [00:00, 150.03it/s]


Fitting 5 folds for each of 53 candidates, totalling 265 fits
A*01:01  -> Probability: 0.9999998876926364	 Final_classification: True	 AUC: 1.0	 Accuracy: 0.8536585365853658	 Precision: 0.625	 Sensitivity: 1.0	 Specificity: 0.8064516129032258


100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 348.56it/s]
8it [00:00, 194.67it/s]
100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 551.49it/s]
8it [00:00, 184.65it/s]


Fitting 5 folds for each of 53 candidates, totalling 265 fits
A*02:01  -> Probability: 0.9999999999999918	 Final_classification: True	 AUC: 0.9782608695652176	 Accuracy: 0.8536585365853658	 Precision: 0.7727272727272727	 Sensitivity: 0.9444444444444444	 Specificity: 0.782608695652174


...

Fitting 5 folds for each of 53 candidates, totalling 265 fits
DRB1*15:01  -> Probability: 0.9999961571641421	 Final_classification: True	 AUC: 1.0	 Accuracy: 0.902439024390244	 Precision: 0.7142857142857143	 Sensitivity: 1.0	 Specificity: 0.8709677419354839


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 56.42it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
8it [00:00, 210.78it/s]
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 67.42it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
8it [00:00, 237.23it/s]


Fitting 5 folds for each of 53 candidates, totalling 265 fits
DRB1*15:02  -> Probability: 0.10581744807806967	 Final_classification: False	 AUC: 0.9560810810810813	 Accuracy: 0.926829268292683	 Precision: 0.6	 Sensitivity: 0.75	 Specificity: 0.945945945945946


0it [00:00, ?it/s]                                        | 0/1 [00:00<?, ?it/s]
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 70.18it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
8it [00:00, 191.48it/s]
0it [00:00, ?it/s]                                        | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 39.22it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
8it [00:00, 203.62it/s]


Fitting 5 folds for each of 53 candidates, totalling 265 fits
DRB1*16:01  -> Probability: 0.20756937443938542	 Final_classification: False	 AUC: 0.9912280701754388	 Accuracy: 0.975609756097561	 Precision: 0.75	 Sensitivity: 1.0	 Specificity: 0.9736842105263158
No HLA-related TCRs have been found in your repertoire
DRB1*16:02  -> Probability: N/A	 Final_classification: False	 AUC: 0.5	 Accuracy: 1	 Precision: 0	 Sensitivity: 0	 Specificity: 1


In [ ]:
17_B	DQB1*02:01	alpha+beta
17_B	DQB1*06:02	alpha+beta
17_B	DQA1*01:02	alpha+beta
17_B	DQA1*05:01	alpha+beta
17_B	A*01:01	alpha+beta
17_B	A*02:01	alpha+beta
17_B	B*07:02	alpha+beta
17_B	B*08:01	alpha+beta
17_B	C*07:01	alpha+beta
17_B	C*07:02	alpha+beta
17_B	DPB1*04:02	alpha+beta
17_B	DPB1*04:02	alpha+beta
17_B	DRB1*03:01	alpha+beta
17_B	DRB1*15:01	alpha+beta

In [9]:
df_final_params[df_final_params['Final_classification']==True]

,HLA,Probability,Final_classification,AUC,Accuracy,Precision,Sensitivity,Specificity,Individual_ID
3,DRB1*15:01,1.0,True,1.0,0.902439,0.714286,1.0,0.870968,17_B
20,DRB1*03:01,0.999969,True,1.0,1.0,1.0,1.0,1.0,17_B
26,DQB1*06:02,0.999964,True,0.996212,0.902439,0.666667,1.0,0.878788,17_B
38,DQB1*02:01,1.0,True,1.0,0.97561,0.909091,1.0,0.967742,17_B
40,DQA1*05:01,1.0,True,1.0,0.97561,0.9,1.0,0.96875,17_B
48,DQA1*01:02,0.999995,True,0.822967,0.804878,0.789474,0.789474,0.818182,17_B
55,DPB1*04:02,1.0,True,0.97619,0.878049,0.545455,1.0,0.857143,17_B
56,DPB1*04:01,0.986079,True,0.904762,0.804878,0.787879,0.962963,0.5,17_B
67,C*07:02,0.999548,True,0.971264,0.878049,0.733333,0.916667,0.862069,17_B
68,C*07:01,0.999996,True,0.890323,0.926829,0.888889,0.8,0.967742,17_B


In [10]:
df_final_params.to_csv('/home/ruiz/example_patient_results.tsv',sep='\t',index=False)

In [2]:
import os
os.getcwd()

'/home/ruiz/Scripts/HLA_Guessr_package'

In [10]:
unique_ps

{(), ('17_B',)}

In [12]:
dm.ps

[[], ['17_B']]